In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval

***
Contents 기반 추천 해보기
***

In [3]:
df = pd.read_csv('wondoofin.csv')
cat_df = pd.read_csv('category.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 559 non-null    int64 
 1   이름                 559 non-null    object
 2   로스터리               559 non-null    object
 3   타입                 559 non-null    object
 4   로스팅 포인트            559 non-null    object
 5   지속가능성              113 non-null    object
 6   컵 노트               559 non-null    object
 7   바디감                559 non-null    int64 
 8   신맛                 559 non-null    int64 
 9   단맛                 559 non-null    int64 
 10  쓴맛                 559 non-null    int64 
 11  커피 소개              559 non-null    object
 12  나라                 543 non-null    object
 13  로스터리 소개            559 non-null    object
 14  로스터리 주소            550 non-null    object
 15  이름.1               559 non-null    object
 16  식품의 유형             554 non-null    object
 1

In [7]:
df.columns

Index(['id', '이름', '로스터리', '타입', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛',
       '쓴맛', '커피 소개', '나라', '로스터리 소개', '로스터리 주소', '이름.1', '식품의 유형',
       '제조원 및 소재지', '유통기한', '제조일자', '내용량', '보관 방법', '원재료 및 함량',
       '제품문의 관련 주소 및 전화번호', '카페인', '로스터리ID'],
      dtype='object')

In [6]:
df.shape

(559, 26)

***
null 값 처리
- 지속가능성 : 113
- 나라 : 543
***
- 로스터리 주소 : 550
- 제조원 및 소재지 : 557/559
- 유통기한 : 557
- 보관 방법 : 545
- 식품의 유형 : 554
***

In [5]:
# 리스트 처리
df['컵 노트'] = df['컵 노트'].apply(literal_eval)
df['컵 노트'][0]

['토피', '맥아', '흑설탕']

***

In [6]:
#################################
########## 지속 가능성 ##########
#################################
for i in range(len(df)):
    if df['지속가능성'][i] is np.nan:
        df.at[i, '지속가능성'] = 0
        
df['지속가능성']

0         0
1         0
2       유기농
3      공정무역
4         0
       ... 
554       0
555       0
556       0
557       0
558       0
Name: 지속가능성, Length: 559, dtype: object

***
나라
***

In [7]:
df['나라'] = df['나라'].str.split(', ')
df['나라'][0]

['브라질', '과테말라']

In [8]:
for i in range(len(df)):
    if df['나라'][i] is np.nan:
        df.at[i, '나라'] = 0
        
df['나라']

0           [브라질, 과테말라]
1                 [세라도]
2                 [아리차]
3             [브라질, 인도]
4              [리무 볼레소]
             ...       
554                   0
555       [에티오피아, 콜롬비아]
556         [브라질, 콜롬비아]
557               [파젠다]
558    [과테말라, 콜롬비아, 인도]
Name: 나라, Length: 559, dtype: object

***
추천에 필요한 값들
1. 카페인 유무 : 카페인 칼럼
2. 블렌드/ 싱글오리진
3. 향 카테고리
4. 맛의 강도 : 쓴맛, 단맛, 바디감, 산미
***

In [24]:
features = pd.get_dummies(df, columns=['타입'], dtype=int)
features.head(2)

,id,이름,로스터리,로스팅 포인트,지속가능성,컵 노트,바디감,신맛,단맛,쓴맛,...,제조일자,내용량,보관 방법,원재료 및 함량,제품문의 관련 주소 및 전화번호,카페인,로스터리ID,타입_디카페인,타입_블렌드,타입_싱글오리진
0,43,데일리스윗,언더프레셔,미디엄다크,NaN,"['토피', '맥아', '흑설탕']",4,2,4,3,...,상단 표기일,140g / 1kg,"직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 보관하여 주십시오.","커피원두 100% (브라질산 60%, 과테말라산 40%)",핸디엄 고객센터 1599-2681,1,57,0,1,0
1,3896,브라질 세라도 디카페인,운조커피,미디엄다크,NaN,['null'],4,2,4,3,...,주문 확인 후 제조,500g 1kg,"직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관",커피원두 100%,운조커피 010-4535-8819,0,67,1,0,0


In [25]:
features.columns

Index(['id', '이름', '로스터리', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛', '쓴맛',
       '커피 소개', '나라', '로스터리 소개', '로스터리 주소', '이름.1', '식품의 유형', '제조원 및 소재지',
       '유통기한', '제조일자', '내용량', '보관 방법', '원재료 및 함량', '제품문의 관련 주소 및 전화번호', '카페인',
       '로스터리ID', '타입_디카페인', '타입_블렌드', '타입_싱글오리진'],
      dtype='object')

In [26]:
features = features[['id', '나라', '로스팅 포인트', '지속가능성', '바디감', '신맛', '단맛', '쓴맛', '타입_디카페인', '타입_블렌드', '타입_싱글오리진']]
features.head(2)

,id,나라,로스팅 포인트,지속가능성,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진
0,43,"브라질, 과테말라",미디엄다크,NaN,4,2,4,3,0,1,0
1,3896,세라도,미디엄다크,NaN,4,2,4,3,1,0,0


In [27]:
features['지속가능성'].unique()

array([nan, '유기농', '공정무역', '직접무역'], dtype=object)

In [28]:
features['로스팅 포인트'].unique()

array(['미디엄다크', '라이트미디엄', '다크', '미디엄', '라이트'], dtype=object)

In [29]:
features = pd.get_dummies(features, columns=['지속가능성', '로스팅 포인트'], dtype=int)
features.head(2)

,id,나라,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진,지속가능성_공정무역,지속가능성_유기농,지속가능성_직접무역,로스팅 포인트_다크,로스팅 포인트_라이트,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크
0,43,"브라질, 과테말라",4,2,4,3,0,1,0,0,0,0,0,0,0,0,1
1,3896,세라도,4,2,4,3,1,0,0,0,0,0,0,0,0,0,1


In [33]:
notes = pd.read_csv('category.csv')
notes = notes.loc[:, notes.columns!='컵 노트']
notes.head(2)

,id,꽃,과일,허브,달콤함,고소함,향료_풍미,초콜릿
0,43,0,0,0,1,0,1,0
1,3896,0,0,0,0,0,0,0


In [35]:
features = pd.merge(features, notes, on="id")
features.head(2)

,id,나라,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진,지속가능성_공정무역,...,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크,꽃,과일,허브,달콤함,고소함,향료_풍미,초콜릿
0,43,"브라질, 과테말라",4,2,4,3,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
1,3896,세라도,4,2,4,3,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [36]:
features.to_csv('features.csv', index=False)

test = pd.read_csv('features.csv')
test.head()

,id,나라,바디감,신맛,단맛,쓴맛,타입_디카페인,타입_블렌드,타입_싱글오리진,지속가능성_공정무역,...,로스팅 포인트_라이트미디엄,로스팅 포인트_미디엄,로스팅 포인트_미디엄다크,꽃,과일,허브,달콤함,고소함,향료_풍미,초콜릿
0,43,"브라질, 과테말라",4,2,4,3,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
1,3896,세라도,4,2,4,3,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3458,아리차,3,4,5,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,22,"브라질, 인도",4,1,3,4,0,1,0,1,...,0,0,0,0,0,0,1,1,0,1
4,180,리무 볼레소,5,3,5,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,1
